In [39]:
import codecs,unidecode,yaml,glob,re
import itertools as it
import pandas as pd

In [40]:
nQuizz=10
nAnnee=24
repQuizz="/Users/gilles/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/"
fQuizzoodle=repQuizz+"QuizzAll.csv"
fAliasQDLE=repQuizz+"Quizz-Alias.yaml"
fNotesOriginales=repQuizz+"%d-Quizz-Notes.xlsx"%nAnnee
fNotesFinales=repQuizz+"%d-Quizz-Notes-Finales.xlsx"%nAnnee
fCM=repQuizz+"CM.csv"

In [41]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Fichier Etudiants

In [42]:
dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8")
dfCM.columns

Index([u'Numéro d’identification', u'Nom de famille', u'Prénom',
       u'Adresse de courriel', u'Dernier téléchargement depuis ce cours'],
      dtype='object')

In [43]:
dfCM.set_index(u"Numéro d’identification",inplace=True)
dfCM[u"Code étudiant"]=dfCM.index.astype(unicode)
dfCM

,Nom de famille,Prénom,Adresse de courriel,Dernier téléchargement depuis ce cours,Code étudiant
Numéro d’identification,,,,,
450977,ALONSO LOPEZ-GRAFFEUIL,ALICE,alice.alonso-lopez-graffeuil@etu.u-bordeaux-mo...,1729585480,450977
448262,ANDRIAMANALINTSOA,HERINIAINA,heriniaina.andriamanalintsoa@etu.u-bordeaux-mo...,1729585480,448262
450285,APAYDIN,MERVAN,mervan.apaydin@etu.u-bordeaux-montaigne.fr,1729585480,450285
441237,ATAM-KASSIGADOU,ELIZA,eliza.atam-kassigadou@etu.u-bordeaux-montaigne.fr,1729585480,441237
445328,AUGEZ,CLEMENT,clement.augez@etu.u-bordeaux-montaigne.fr,1729585480,445328
451963,AVENTURIN,JADE,jade.aventurin@etu.u-bordeaux-montaigne.fr,1729585480,451963
450988,BAPTISTE LAMBERT,REYHNAEL,reyhnael.baptiste-lambert@etu.u-bordeaux-monta...,1729585480,450988
416094,BEL,ELENA-ROSE,elena-rose.bel@etu.u-bordeaux-montaigne.fr,1729585480,416094
450742,BELVISI,ELOISE,eloise.belvisi@etu.u-bordeaux-montaigne.fr,1729585480,450742


# Fichier Alias

In [44]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inputfile:
        correspondances=yaml.safe_load(inputfile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'A.verdel': ['447703'],
 'ANDHERIN': ['448262'],
 'ANDRIA.HERIN': ['448262'],
 'Alex.verdel': ['447703'],
 'Alice Alonso': ['450977'],
 'Alix Lonlas': ['450067'],
 u'Am\xe9lie Lauilh\xe9': ['448793'],
 'Ana PIOTON': ['451262'],
 'Ana Pioton': ['451262'],
 u'Ana\xe9': ['438682'],
 u'Ana\xe9 Lamy': ['438682'],
 'Andria.Heriniaina': ['448262'],
 'Apay merv': ['450285'],
 'Apaydin Mervan': ['450285'],
 'Bel Elena-Rose': ['416094'],
 'Bel elena': ['416094'],
 'Belvisi Eloise': ['450742'],
 'Blanche Navarro': ['451482'],
 'Bouffenie marylou': ['446920'],
 u'CL\xc9MENT.augez': ['445328'],
 'Camilla Sulejmani': ['402804'],
 'Camille Etcheverry': ['446690'],
 'Charlotte Durocher': ['444843'],
 'Cheick GUIRA': ['451579'],
 'Chlo Roub': ['446964'],
 u'Chlo\xe9 Roubert': ['446964'],
 u'Chlo\xe9 Roubert Dieuaide': ['446964'],
 'Choquet Clara': ['448189'],
 'Choquet clara': ['448189'],
 'Cla cho': ['448189'],
 'Clara Choquet': ['448189'],
 'Clara choquet': ['448189'],
 u'Cl\xe9ment.Augez': ['445328

In [45]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
# colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(fQuizzoodle,sep=";",encoding="utf8",header=0)
    lQuizz.columns=["pseudo"]+lQuizz.columns.tolist()[1:]
    # lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [46]:
# dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
lQuizz

,pseudo,0(2),1(4),2(6),3(9),4(11),5(13),6(16),7(18),8(20),...,20(48),21(51),22(53),23(55),24(58),25(60),26(62),27(65),28(67),29(69)
0,,E,F,G,B,A,D,B,A,A,...,D,B,A,D,B,C,D,D,B,A
1,Qinyan Zhang,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2,ClémentineLagneaux,?,?,?,?,?,?,?,?,?,...,C,D,B,C,B,C,D,B,?,D
3,Romane Lebeau,?,?,?,?,?,?,?,?,?,...,D,A,A,B,B,C,?,B,A,A
4,Camilla Sulejmani,?,?,?,?,?,?,?,?,?,...,D,B,A,B,C,C,C,B,B,?
5,Eloise Belvisi,?,?,?,?,?,B,D,B,B,...,D,A,D,D,D,C,B,D,C,D
6,Sara Le Gall,?,?,?,?,?,D,A,A,B,...,?,?,?,?,?,?,?,?,?,?
7,joy.lscr,?,?,?,?,A,D,B,A,A,...,?,?,?,?,?,?,?,?,?,?
8,Qinyan ZHANG,E,E,G,B,D,D,C,?,B,...,D,B,A,B,B,?,D,?,?,?
9,DUBARRE Anaïs,E,F,G,D,A,D,A,A,B,...,B,B,A,D,C,C,D,A,C,A


In [47]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()

In [48]:
lQuizz

,pseudo,0(2),1(4),2(6),3(9),4(11),5(13),6(16),7(18),8(20),...,20(48),21(51),22(53),23(55),24(58),25(60),26(62),27(65),28(67),29(69)
0,,E,F,G,B,A,D,B,A,A,...,D,B,A,D,B,C,D,D,B,A
1,Qinyan Zhang,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2,ClémentineLagneaux,?,?,?,?,?,?,?,?,?,...,C,D,B,C,B,C,D,B,?,D
3,Romane Lebeau,?,?,?,?,?,?,?,?,?,...,D,A,A,B,B,C,?,B,A,A
4,Camilla Sulejmani,?,?,?,?,?,?,?,?,?,...,D,B,A,B,C,C,C,B,B,?
5,Eloise Belvisi,?,?,?,?,?,B,D,B,B,...,D,A,D,D,D,C,B,D,C,D
6,Sara Le Gall,?,?,?,?,?,D,A,A,B,...,?,?,?,?,?,?,?,?,?,?
7,joy.lscr,?,?,?,?,A,D,B,A,A,...,?,?,?,?,?,?,?,?,?,?
8,Qinyan ZHANG,E,E,G,B,D,D,C,?,B,...,D,B,A,B,B,?,D,?,?,?
9,DUBARRE Anaïs,E,F,G,D,A,D,A,A,B,...,B,B,A,D,C,C,D,A,C,A


In [49]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

In [51]:
for nP,pseudo in enumerate(pseudos):
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudo]=idPseudo

Nom a chercher:
 


In [28]:
# pseudo=u"Cotentin, Pailhes"
# alias[pseudo]=["414180","411916"]

In [52]:
sPseudo="pailh"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [53]:
pn=dfCM["PN"].tolist()
np=dfCM["NP"].tolist()

In [54]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]

In [55]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [56]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'A.verdel': ['447703'],
  'ANDHERIN': ['448262'],
  'ANDRIA.HERIN': ['448262'],
  'Alex.verdel': ['447703'],
  'Alice Alonso': ['450977'],
  'Alix Lonlas': ['450067'],
  u'Am\xe9lie Lauilh\xe9': ['448793'],
  'Ana PIOTON': ['451262'],
  'Ana Pioton': ['451262'],
  u'Ana\xe9': ['438682'],
  u'Ana\xe9 Lamy': ['438682'],
  'Andria.Heriniaina': ['448262'],
  'Apay merv': ['450285'],
  'Apaydin Mervan': ['450285'],
  'Bel Elena-Rose': ['416094'],
  'Bel elena': ['416094'],
  'Belvisi Eloise': ['450742'],
  'Blanche Navarro': ['451482'],
  'Bouffenie marylou': ['446920'],
  u'CL\xc9MENT.augez': ['445328'],
  'Camilla Sulejmani': ['402804'],
  'Camille Etcheverry': ['446690'],
  'Charlotte Durocher': ['444843'],
  'Cheick GUIRA': ['451579'],
  'Chlo Roub': ['446964'],
  u'Chlo\xe9 Roubert': ['446964'],
  u'Chlo\xe9 Roubert Dieuaide': ['446964'],
  'Choquet Clara': ['448189'],
  'Choquet clara': ['448189'],
  'Cla cho': ['448189'],
  'Clara Choquet': ['448189'],
  'Clara choquet

In [57]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Bonnes réponses pour les différents quizz du QuizzAll
- sur la deuxième ligne lQuizz[1], il y a toutes les bonnes réponses
- on coupe le Quizz All par groupe de trois questions pour chaque quizz d'origine
 - *quizzReponses*

In [58]:
with codecs.open(fQuizzoodle,"r",encoding="utf8") as input:
    lQuizz=input.readlines()

In [59]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]    

In [60]:
def getReponses(ligne):
    result={}
    ligneEls=[l.strip() for l in ligne.split(";")]
    pseudo=ligneEls[0]
    reponses=ligneEls[1:]
    for num,rep in enumerate(chunks(reponses,3)):
        result["%02d"%(num+2)]=rep
    return pseudo,result    

In [61]:
temp,quizzReponses=getReponses(lQuizz[1])
quizzReponses

{'02': [u'E', u'F', u'G'],
 '03': [u'B', u'A', u'D'],
 '04': [u'B', u'A', u'A'],
 '05': [u'B', u'C', u'D'],
 '06': [u'D', u'D', u'C'],
 '07': [u'B', u'A', u'C'],
 '08': [u'C', u'A', u'D'],
 '09': [u'B', u'A', u'D'],
 '10': [u'B', u'C', u'D'],
 '11': [u'D', u'B', u'A']}

In [62]:
def markReponses(reponses):
    scores={}
    for q in quizzReponses:
#         print q
        score=0
        for n,r in enumerate(quizzReponses[q]):
#             print n,r,reponses[q][n]
            if r==reponses[q][n]:
                score+=1
        scores[q]=int(round(float(score)*20/3,0))    
    return scores

# Notes du pseudo pour les différents quizz
- sur la ligne du pseudo, on a toutes les réponses
- on coupe les réponses par groupe de trois réponses
- on compare avec les bonnes réponses

In [63]:
alias[u"mon-a sal-a"]

KeyError: u'mon-a sal-a'

In [64]:
quizzNotes={}
for l in lQuizz[2:]:
    nom,reponses=getReponses(l)
#     print nom,reponses
    
    marks=markReponses(reponses)
    
#     print nom, " ; ".join([str(marks[k]) for k in sorted(marks.keys())])
    
    if nom in alias:
#         print nom,alias[nom]
        akas=alias[nom]
    else:
        print "pas de",nom
        akas=[]
        
    for aka in akas:
        if aka in quizzNotes:
            print aka,"déjà dans les notes",invAlias[aka]
            print quizzNotes[aka]
            print marks
            for k,v in quizzNotes[aka].iteritems():
                if v<marks[k]:
                    quizzNotes[aka][k]=marks[k]
            print quizzNotes[aka]
        else:
            quizzNotes[aka]=marks
print 
print "===================="
print
for aka in sorted(quizzNotes.keys()):
#    print chercherIdentifiant(aka)
    print aka, ", ".join([str(quizzNotes[aka][k]) for k in sorted(quizzNotes[aka].keys())])

449046 déjà dans les notes ['Qinyan ZHANG', 'Qinyan Zhang', 'Qinyan-Zhang']
{'02': 0, '03': 0, '06': 0, '07': 0, '04': 0, '05': 0, '08': 0, '09': 0, '11': 0, '10': 0}
{'02': 13, '03': 13, '06': 7, '07': 7, '04': 0, '05': 13, '08': 20, '09': 13, '11': 0, '10': 13}
{'02': 13, '03': 13, '06': 7, '07': 7, '04': 0, '05': 13, '08': 20, '09': 13, '11': 0, '10': 13}


402804 0, 0, 0, 7, 13, 7, 7, 13, 7, 7
413294 0, 0, 0, 0, 7, 0, 7, 0, 20, 0
416094 20, 20, 7, 13, 13, 0, 13, 13, 13, 7
431171 7, 20, 0, 7, 7, 0, 7, 13, 0, 7
433462 20, 20, 7, 20, 7, 13, 20, 13, 13, 7
435763 7, 13, 0, 13, 13, 0, 7, 13, 0, 13
436744 13, 20, 7, 20, 7, 0, 13, 7, 7, 7
438087 13, 20, 20, 7, 7, 7, 7, 13, 7, 7
438682 13, 7, 0, 7, 13, 0, 13, 13, 20, 0
444338 13, 0, 7, 13, 7, 0, 13, 13, 20, 7
445098 20, 13, 7, 13, 13, 0, 7, 20, 13, 13
445328 20, 7, 0, 13, 7, 0, 13, 7, 13, 0
445501 13, 7, 13, 7, 7, 7, 0, 0, 7, 0
445918 20, 20, 0, 20, 13, 0, 20, 0, 7, 0
446480 20, 13, 0, 7, 13, 0, 7, 7, 0, 0
446664 20, 13, 0, 7, 13, 0, 7, 13,

In [65]:
quizzNotes[u"443367"]

KeyError: u'443367'

In [93]:
dfNotesOriginales=pd.read_excel(fNotesOriginales)
dfNotesOriginales[u"Numéro d’identification"]=dfNotesOriginales[u"Numéro d’identification"].astype("int").astype("str")
dfNotesOriginales[["Nom"]+["Q%02d"%i for i in range(2,12)]]

,Nom,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11
0,ALONSO LOPEZ-GRAFFEUIL,-,-,-,-,11,NaN,13,15,12,-
1,ANDRIAMANALINTSOA,13,14,14,16,14,NaN,13,13,13,10
2,APAYDIN,14,13,16,19,11,NaN,10,15,17,10
3,ATAM-KASSIGADOU,-,-,-,-,-,NaN,-,-,-,-
4,AUGEZ,14,13,11,20,13,NaN,12,15,13,12
5,AVENTURIN,14,11,13,16,18,NaN,13,13,18,12
6,BAPTISTE LAMBERT,-,-,-,-,-,NaN,-,-,-,-
7,BEL,-,16,16,20,-,NaN,13,12,15,18
8,BELVISI,17,14,16,19,-,NaN,12,15,13,15
9,BERTEL--BOUSBAINE,14,14,20,19,-,NaN,13,13,12,15


In [86]:
# dfCampus=dfNotesOriginales[[u"Numéro d'identification"]+u"Nom Prénom".split(" ")+["Quizz%02d (Brut)"%(i+1) for i in range(1,10)]]
dfCampus=dfNotesOriginales[[u"Numéro d’identification"]+u"Nom Prénom".split(" ")+["Q%02d"%(i+1) for i in range(1,11)]]
dfCampus.set_index(u"Numéro d’identification",inplace=True)
# dfCampus.loc["417001"]

In [94]:
nouvellesNotes={}
for etu in quizzNotes:
    print etu
    if etu in dfCampus.index:
        for q in quizzNotes[etu]:
            # if quizzNotes[etu][q] > dfCampus.loc[etu,"Quizz%s (Brut)"%q] or dfCampus.loc[etu,"Quizz%s (Brut)"%q]=="-":
            if quizzNotes[etu][q] > dfCampus.loc[etu,"Q%s"%q] or dfCampus.loc[etu,"Q%s"%q]=="-" or dfCampus.loc[etu,"Q%s"%q]!=dfCampus.loc[etu,"Q%s"%q]:
                # print u"Quizz %s"%q,dfCampus.loc[etu,"Quizz%s (Brut)"%q],"=>",quizzNotes[etu][q]
                print u"Quizz %s"%q,dfCampus.loc[etu,"Q%s"%q],"=>",quizzNotes[etu][q]
                if not etu in nouvellesNotes:
                    nouvellesNotes[etu]={}
                nouvellesNotes[etu][q]=quizzNotes[etu][q]
                # dfCampus.loc[etu,"Quizz%s (Brut)"%q]=quizzNotes[etu][q]
                dfCampus.loc[etu,"Q%s"%q]=quizzNotes[etu][q]
    else:
        print "PB", etu, "pas de notes originales"
        print quizzNotes[etu]
        nouvellesNotes[etu]=quizzNotes[etu]
len(nouvellesNotes)

451638
Quizz 07 nan => 7
436744
Quizz 07 nan => 0
416094
Quizz 07 nan => 0
447689
Quizz 07 nan => 0
450517
Quizz 07 nan => 7
447738
Quizz 07 nan => 0
445098
Quizz 07 nan => 0
446920
Quizz 07 nan => 0
447289
Quizz 07 nan => 0
445328
Quizz 07 nan => 0
451148
Quizz 07 nan => 0
450742
Quizz 07 nan => 7
448247
Quizz 07 nan => 0
448262
Quizz 07 nan => 7
431171
Quizz 07 nan => 0
433462
Quizz 07 nan => 13
445501
Quizz 07 nan => 7
450554
Quizz 07 nan => 0
447962
Quizz 07 nan => 7
413294
Quizz 07 nan => 0
451471
Quizz 07 nan => 7
438087
Quizz 07 nan => 7
446682
Quizz 07 nan => 7
435763
Quizz 07 nan => 0
449046
Quizz 07 nan => 7
447655
Quizz 07 nan => 0
450546
Quizz 07 nan => 13
447984
Quizz 07 nan => 0
445918
Quizz 07 nan => 0
450937
Quizz 07 nan => 7
447541
Quizz 07 nan => 7
449093
Quizz 07 nan => 7
448621
Quizz 07 nan => 7
402804
Quizz 07 nan => 7
447697
Quizz 07 nan => 20
447019
Quizz 07 nan => 13
446746
Quizz 07 nan => 13
447703
Quizz 07 nan => 0
446480
Quizz 07 nan => 0
450446
Quizz 07 nan 

59

In [95]:
dfCampus

,Nom,Prénom,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11
Numéro d’identification,,,,,,,,,,,,
450977,ALONSO LOPEZ-GRAFFEUIL,ALICE,-,-,-,-,11,NaN,13,15,12,-
448262,ANDRIAMANALINTSOA,HERINIAINA,20,20,14,16,14,7.0,13,13,13,10
450285,APAYDIN,MERVAN,20,20,16,19,13,7.0,20,15,17,10
441237,ATAM-KASSIGADOU,ELIZA,-,-,-,-,-,NaN,-,-,-,-
445328,AUGEZ,CLEMENT,20,13,11,20,13,0.0,13,15,13,12
451963,AVENTURIN,JADE,14,11,13,16,18,NaN,13,13,18,12
450988,BAPTISTE LAMBERT,REYHNAEL,-,-,-,-,-,NaN,-,-,-,-
416094,BEL,ELENA-ROSE,20,20,16,20,13,0.0,13,13,15,18
450742,BELVISI,ELOISE,17,14,16,19,0,7.0,12,15,13,15


In [96]:
numId="449233"
dfCampus.loc[numId],dfNotesOriginales[dfNotesOriginales[u"Numéro d’identification"]==numId]

(Nom       VIGOUROUX
 Prénom        LOANE
 Q02              14
 Q03              11
 Q04              18
 Q05              19
 Q06              13
 Q07             NaN
 Q08               -
 Q09               -
 Q10               -
 Q11               -
 Name: 449233, dtype: object,
     Numéro d’identification        Nom Prénom  \
 113                  449233  VIGOUROUX  LOANE   
 
                              Adresse de courriel  \
 113  loane.vigouroux@etu.u-bordeaux-montaigne.fr   
 
     Kalaba de Prasi (Contrôle continu - Max) (Brut)  \
 113                                               -   
 
     Test Kalaba de Yomig (Contrôle continu 2024 - Version) (Brut)  \
 113                                                  -              
 
     Test Kalaba de Yomig (Contrôle continu 2024 — Thème) (Brut)  \
 113                                                  -            
 
     Test Kalaba de Kartin (Contrôle continu 2024 — Version) (Brut)  \
 113                                       

In [97]:
dfCampus.to_excel(fNotesFinales)